# Proyecto- Análisis de mercado inmobiliario



## Problema

Recientemente te has incorporado al equipo de Datos de una gran inmobiliaria. La primera tarea que se te asigna es ayudar a los tasadores/as a valuar las propiedades, ya que es un proceso difícil y, a veces, subjetivo. Para ello, propones crear un modelo de Machine Learning que, dadas ciertas características de la propiedad, prediga su precio de venta.

### 1. Pensando como un/a Data Scientist

Responde la siguientes pregunta:
1. ¿Qué datos crees que te ayudarían a trabajar en el problema?¿Por qué?

**Importante**: NO deberás buscar esos datos, solamente justificar qué información crees que te ayudaría a resolver la problemática planteada.



*   Metros cuadrados del terreno: Mientras más metros cuadrados, más valor va a tener la propiedad.

*   Posición del terreno con respecto a zonas de interés: Si es una zona alejada de cualquier punto de interés perderá su valor, de la misma manera que una propiedad en el centro tendrá un valor alto.

*   Materiales de construcción: La variación de los tipos de materiales usados para la construcción es uno de los factores más críticos ya que existen materiales muy poco resistentes en construcciones precarias por un lado y materiales resistentes por el otro.

*   Cantidad de ambientes: Mientras más ambientes tenga construídos su valor será mayor siempre y cuando mantengan una cantidad de metros cuadrado razonables por ambiente.


*   Instalaciones de servicios básicos (Agua,luz,etc): Estas instalaciones cuestan y por lo tanto que ya vengan con la propiedad indica una ventaja, de acuerdo a cuantos servicios vienen instalados su precio será mayor si tiene más y menor si tiene menos o ninguno.








# **1.Análisis exploratorio**

***1.1 Importando librerías necesarias para el análisis.***

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from pandas import DataFrame

***1.2 Cargando el dataset y viendo el formato de los datos a analizar.***

In [ ]:
data_properties = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/DS_Proyecto_01_Datos_Properati.csv", 
                        error_bad_lines=False)

In [ ]:
print(" Cantidad de columnas:",len(data_properties.columns),"\n","Cantidad de filas:",len(data_properties.index))

In [ ]:
data_properties.head()

***1.3 Se estudian los valores faltantes.***

In [ ]:
print(data_properties.info())

In [ ]:

print(data_properties.isna().sum())

Se puede reconocer que sólo la latitud, longitud, baños y variables de superficie son las características que poseen datos faltantes.

***1.4 Analisis de tipo de propiedad.***

In [ ]:
data_properties.groupby('property_type').count()

In [ ]:

data_properties['property_type'].value_counts()


In [ ]:
plt.subplots(figsize=(15,6))
sns.countplot(data = data_properties, x = "property_type",order = data_properties["property_type"].value_counts().index)
plt.xlabel("Tipo")
plt.ylabel("Cantidad")
plt.show()

Se observa en el gráfico como por una considerable diferencia, las propiedades de tipo departamento, casa y PH son las más populares y por lo tanto serían las principales a tomar en cuenta para nuestro modelo.

***1.5 Análisis de región de propiedad.***

In [ ]:
plot_region_l3 = data_properties['l3'].value_counts()
plot_region_l2 = DataFrame(data_properties,columns=["l2"])


plt.rcParams.update({'font.size': 20})

plt.subplot(2, 1, 1)
my_labels = plot_region_l2.l2.unique()
plt.pie(plot_region_l2.value_counts(),labels=my_labels,autopct='%1.1f%%')
plt.title("Localidad")
plt.axis('equal')

plot_region_l3 = data_properties['l3'].value_counts()
plt.subplot(2, 1, 2)
plot_region_l3.plot(kind="bar")
plt.rcParams["figure.figsize"]=(30, 20)
plt.title("Barrios")
plt.xlabel("Nombre del barrio")
plt.ylabel("Propiedades por barrio")

plt.show()

Teniendo en cuenta las distintas regiones, se puede ver como el 63% de los datos de las propiedades que poseemos son de Capital Federal, lo cual convierte a esta región en una variable importante a considerar al momento de filtrar los datos para nuestro modelo.

***1.6. Filtro de características relevantes***

En el siguiente código se determina el rango de variables más importantes a considerar. En este caso se determina un rango de los tres tipos de propiedades más importantes en conjunto con la región con más cantidad de propiedades.

In [ ]:
cant_topTipos=3
cant_topRegion=1

In [ ]:
top_tipos=data_properties["property_type"].value_counts().head(cant_topTipos).index.tolist()

In [ ]:
top_regiones=data_properties["l2"].value_counts().head(cant_topRegion).index.tolist()

In [ ]:
data_top_properties = data_properties[data_properties.property_type.isin(top_tipos) & data_properties.l2.isin(top_regiones)]

In [ ]:
data_top_properties

***1.7 Filtro de outliers***

**1.7.1 Análisis de valores generales:** Analizamos los valores generales de todas las características para usar como guía y graficamos para ver cuales variables cuentan con outliers y que tan distantes son para saber cuanto afectan a nuestro análisis. 

In [ ]:
data_top_properties.describe()

In [ ]:
comp_param = ["property_type","price","surface_covered","surface_total","rooms","bedrooms","bathrooms"]

In [ ]:
plt.rcParams.update({'font.size': 10})
figu= plt.figure()

graph=sns.pairplot(data_top_properties[comp_param],hue="property_type")
graph.fig.set_size_inches(16,8)
plt.grid()
plt.show()

In [ ]:
plt.subplot(2, 1, 1)
sns.boxplot(data=data_top_properties,x="surface_total")

plt.subplot(2, 1, 2)
sns.boxplot(data=data_top_properties,x="surface_covered")
plt.show()

In [ ]:
sns.scatterplot(x="surface_total",y="surface_covered", hue="property_type" , data=data_top_properties)

Se puede ver a plena vista como a partir de valores mayores a 20.000 en superficies totales y cubiertas ya podrían ser considerados outliers pero para un análisis más preciso se recurrirá al Z-Score y el IQR.

**1.7.2 Análisis Z-score y IQR.**

**Z-Score:** Analizo los distintos valores de Z-score para tener como referencia además del IQR haciendo comparación en cual sería la efectividad de cada uno en caso de tomar como filtro de outliers.

In [ ]:
data_top_properties_2 = data_top_properties[comp_param].copy()

In [ ]:
outliers=[]
def detect_outliers(data):

  threshold = 3
  mean = np.mean(data)
  std = np.std(data)

  for i in data: 
    z_score = (i-mean)/std
    if np.abs(z_score) > threshold:
      outliers.append(i)
  return outliers

In [ ]:
outlier_surface_total=detect_outliers(data_top_properties_2["surface_total"])
outlier_surface_covered=detect_outliers(data_top_properties_2["surface_covered"])
outlier_price=detect_outliers(data_top_properties_2["price"])
print("Outliers de total de superficie:",outlier_surface_total)
print("Outliers de superficie cubierta:",outlier_surface_covered)
print("Outliers de precio:",outlier_price)

**IQR:** Considero que el filtro de IQR es más preciso al momento de filtrar por cuartiles.

In [ ]:
def detect_IQR(data):
  Q1=data_top_properties_2[data].quantile(0.25)
  Q3=data_top_properties_2[data].quantile(0.75)
  IQR= Q3-Q1
  print("Superficie maxima:",data_top_properties_2[data].max())
  print("Superficie minima:",data_top_properties_2[data].min())
  print("IQR:",IQR)



In [ ]:
print("Superficie total")
print("----------------------")
detect_IQR("surface_total")

In [ ]:
print("Superficie cubierta")
print("----------------------")
detect_IQR("surface_covered")

In [ ]:
print("Precio")
print("----------------------")
detect_IQR("price")

**1.7.3 Aplico el filtro de IQR.**


In [ ]:
def IQR_filter(dataf,dataname):
    q1 = dataf[dataname].quantile(0.25)
    q3 = dataf[dataname].quantile(0.75)
    iqr = q3 - q1
    fence_low = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    cleaned_data = dataf.loc[(dataf[dataname] > fence_low) & (dataf[dataname] < fence_high)]
    return cleaned_data

In [ ]:
properties_filtered = IQR_filter(data_top_properties_2,"surface_total")
properties_filtered = IQR_filter(properties_filtered,"surface_covered")
properties_filtered = IQR_filter(properties_filtered,"price")
properties_filtered = properties_filtered[properties_filtered.surface_total > properties_filtered.surface_covered]

*Compruebo que los outliers fueron filtrados*

In [ ]:
print("Superficie total máxima")
print("----------------------")

print(properties_filtered["surface_total"].max(),"\n")
print("Superficie cubierta máxima")
print("----------------------")

print(properties_filtered["surface_covered"].max(),"\n")
print("Precio máximo")
print("----------------------")

print(properties_filtered["price"].max(),"\n")

In [ ]:
print("Máximo de superficie total:", properties_filtered["surface_total"].max())
print("Máximo de superficie cubierta:", properties_filtered["surface_covered"].max())

plt.subplot(2, 1, 1)
sns.boxplot(data=properties_filtered,x="surface_total")

plt.subplot(2, 1, 2)
sns.boxplot(data=properties_filtered,x="surface_covered")
plt.show()

In [ ]:
plt.rcParams.update({'font.size': 10})
figu= plt.figure()

graph=sns.pairplot(properties_filtered[comp_param],hue="property_type")
graph.fig.set_size_inches(16,8)
plt.grid()
plt.show()

Ahora el análisis visual es más fácil de hacer, se pueden reconocer agrupaciones de valores por secciones, aunque la abundancia de datos dispersos puede aun dificultar el análisis.

**1.7.4 Correlaciones:** Los valores más correlacionados con el precio incluyen la superficie cubierta , la superficie total y la cantidad de cuartos.

In [ ]:
corr = properties_filtered.corr()

plt.figure(figsize=(8,8))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
           cmap= 'coolwarm')
plt.xticks(rotation = 45)
plt.yticks(rotation = 45)
plt.show()


# **2. Desafío**

Con los datos obtenidos hasta ahora se puede analizar con más profundidad los datos de latitud y longitud ayudando a aclarar una pregunta: *¿Hay relación entre ubicación y precio?*

*2.1 Instalo los paquetes e importo las librerías necesarias*

In [ ]:
!pip install geopandas

In [ ]:
!pip install geoplot

In [ ]:
import descartes
import mapclassify as mc
from shapely.geometry import Point, Polygon

import geoplot as gplt
import geoplot.crs as gcrs
import geopandas as gpd

%matplotlib inline

In [ ]:
data_map = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DS_Proyecto_01_Datos_Properati.csv", 
                        error_bad_lines=False)

*2.2 Realizo un dataframe para poder graficar las latitudes y longitudes como puntos*

In [ ]:
filter_type= ["Casa","Departamento","PH"]
filter_reg= ["Capital Federal"]
data_map = data_map[data_map.property_type.isin(filter_type)]
data_map = data_map[data_map.l2.isin(filter_reg)]
mask1 = (data_map.surface_total <= 1000) & (data_map.surface_total >= 15) & (data_map.price <= 4000000)
data_map = data_map[mask1]
data_map = data_map.loc[:,["rooms","bedrooms","bathrooms","surface_total","surface_covered","price","lat","lon","property_type"]]
data_map.dropna(inplace=True)
print(data_map.shape)
data_map

In [ ]:
crs = {"init": "EPSG:4326"}

In [ ]:
geometry = [Point(xy) for xy in zip(data_map["lon"],data_map["lat"])]
geometry[:3]

In [ ]:
geo_df = gpd.GeoDataFrame(data_map, crs=crs, geometry = geometry)
geo_df.head()

In [ ]:
gplt.pointplot(geo_df, hue="price", legend=True)

Teniendo los datos visuales se puede ver que hay zonas que se considerarían céntricas, cerca de la costa, que podrían establecer una relación entre posición y precio, siendo los precios más bajos los más alejados de esa posición.

# **3. Machine Learning**



**3.1 Filtrando el dataset.**

In [ ]:
data_ml = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/DS_Proyecto_01_Datos_Properati.csv", 
                        error_bad_lines=False)

In [ ]:
filter_type= ["Casa","Departamento","PH"]
filter_reg= ["Capital Federal"]
data_ml = data_ml[data_ml.property_type.isin(filter_type)]
data_ml = data_ml[data_ml.l2.isin(filter_reg)]
mask1 = (data_ml.surface_total <= 1000) & (data_ml.surface_total >= 15) & (data_ml.price <= 4000000)
data_ml = data_ml[mask1]
data_ml = data_ml.loc[:,["rooms","bedrooms","bathrooms","surface_total","surface_covered","price"]]
data_ml.dropna(inplace=True)
print(data_ml.shape)

**3.2 Métricas**

Para elegir las métricas consideré las más comunes a utilizar haciendo tanto métricas de evaluación sobre errores (RMSE,MAE) como de desempeño (R2):

*   Mean Squared Error (MSE)
*   Root Mean Squared Error (RMSE)
*   Mean Absolute Error (MAE)
*   R2

Se utilizará el error cuadrático medio que representa a la raíz cuadrada de la distancia cuadrada promedio entre el valor real y el valor pronosticado.Indica el ajuste absoluto del modelo a los datos, cuán cerca están los puntos de datos observados de los valores predichos del modelo.
Por último el error cuadrático medio no es del todo intuitivo porque nos da el error medio al cuadrado. Así que si el valor de bienes inmuebles tiene un error cuadrático medio de 1.000.000 parece que tiene mucho error cuando en realidad sería la raíz de 1000 en la media.
Por este motivo uso como referencia el RMSE que saca la raíz del MSE.


**3.3 Recursos necesarios para el modelo**

In [ ]:
import math
from math import sqrt

from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn import neighbors
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression


from sklearn.svm import SVR
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import preprocessing
from sklearn.feature_selection import RFE


*3.3.1 Variables predictoras(X) y a predecir(y)*

In [ ]:
X = data_ml [["surface_total","surface_covered","bathrooms","bedrooms"]]
y = data_ml ["price"]

*3.3.2 Train/test de datos*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

**3.4 Benchmark:** Regresión lineal.

In [ ]:
bench = LinearRegression()
bench.fit(X_train, y_train)

In [ ]:
y_bench_train_pred = bench.predict(X_train)
y_bench_test_pred = bench.predict(X_test)

*3.4.1 Evaluación de benchmark.*

*Creo una función para evaluar el modelo ya que el código se repetirá varias veces*

In [ ]:
def metricas(y_train_pred,y_test_pred):
  MSE_train = mean_squared_error(y_train, y_train_pred)
  MSE_test = mean_squared_error(y_test, y_test_pred)

  RMSE_train = mean_squared_error(y_train, y_train_pred, squared=False)
  RMSE_test = mean_squared_error(y_test, y_test_pred, squared=False)

  MAE_train = mean_absolute_error(y_train, y_train_pred)
  MAE_test = mean_absolute_error(y_test, y_test_pred)

  R2_train = r2_score(y_train, y_train_pred)
  R2_test = r2_score(y_test, y_test_pred)

  print("MSE de train:",MSE_train)
  print("MSE de test:",MSE_test)
  print("Diferencia:",MSE_train-MSE_test)
  print(" ")
  print("RMSE de train:",RMSE_train)
  print("RMSE de test:",RMSE_test)
  print("Diferencia:",RMSE_train-RMSE_test)
  print(" ")
  print("MAE de train:",MAE_train)
  print("MAE de test:",MAE_test)
  print("Diferencia:",MAE_train-MAE_test)
  print(" ")
  print("R2 de train:",R2_train)
  print("R2 de test:",R2_test)
  print("Diferencia:",R2_train-R2_test)
  print(" ")



In [ ]:
metricas(y_bench_train_pred,y_bench_test_pred)

**3.5 Modelos de vecinos más cercanos y árbol de decisión.**



In [ ]:
regressor_knn = KNeighborsRegressor()
regressor_tree = DecisionTreeRegressor()

regressor_knn.fit(X_train, y_train)
regressor_tree.fit(X_train, y_train)

y_tree_pred_train = regressor_tree.predict(X_train)
y_knn_pred_train = regressor_knn.predict(X_train)

y_tree_pred_test = regressor_tree.predict(X_test)
y_knn_pred_test = regressor_knn.predict(X_test)

*3.5.1 Evaluación de los modelos*

In [ ]:
print("Evaluación del árbol de decisiones:","\n")
metricas(y_tree_pred_train,y_tree_pred_test)

In [ ]:
print("Evaluación del modelo de vecinos cercanos:","\n")

metricas(y_knn_pred_train,y_knn_pred_test)

**3.6 Mejorando parámetros de modelos.**

*VECINOS MÁS CERCANOS*

In [ ]:
rmse_val_train_knn = []
rmse_val_test_knn = [] 
scores_knn = []
k_range = range(1,22)
for K in k_range:
    K = K+1

    regressor_knn_3=KNeighborsRegressor(n_neighbors=K)
    regressor_knn_3.fit(X_train,y_train)
    scores_knn.append(regressor_knn_3.score(X_test,y_test))
    y_knn_pred_train_3 = regressor_knn_3.predict(X_train)
    y_knn_pred_test_3 = regressor_knn_3.predict(X_test)

    RMSE_train_3 = mean_squared_error(y_train, y_knn_pred_train_3, squared=False)
    RMSE_test_3 = mean_squared_error(y_test, y_knn_pred_test_3, squared=False)


    rmse_val_train_knn.append(RMSE_train_3)
    rmse_val_test_knn.append(RMSE_test_3)


In [ ]:

curve_train_knn = pd.DataFrame(rmse_val_train_knn)
curve_test_knn = pd.DataFrame(rmse_val_test_knn)

fig = plt.figure(figsize = (15,4))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

ax1.scatter(k_range,scores_knn,marker="o")
ax2.scatter(k_range,rmse_val_train_knn, marker = "o",label="train")
ax2.scatter(k_range,rmse_val_test_knn, marker = "o",label="test")
ax3.plot(curve_train_knn)
ax4.plot(curve_test_knn)

ax1.set_xticks(k_range)
ax2.set_xticks(k_range)

plt.show()



*Eligiendo número de vecinos:*
A partir de un número de vecinos de 12 se empieza a estabilizar por más cambio en el número de vecinos haya por lo cual consideraría óptimo ese valor.


In [ ]:
curve_train_knn

In [ ]:
curve_test_knn

*Evaluando modelo mejorado*

In [ ]:
k= 12
regressor_knn_2=KNeighborsRegressor(n_neighbors=k)
regressor_knn_2.fit(X_train,y_train)
y_knn_pred_train_2 = regressor_knn_2.predict(X_train)
y_knn_pred_test_2 = regressor_knn_2.predict(X_test)

In [ ]:
print("Evaluación del modelo de vecinos cercanos:","\n")

metricas(y_knn_pred_train_2,y_knn_pred_test_2)

*PROFUNDIDAD DE ÁRBOL*

In [ ]:
rmse_val_train_tree = []
rmse_val_test_tree = [] 
scores_tree = []
k_range = range(1,22)
for K in k_range:
    K = K+1

    regressor_tree_3=DecisionTreeRegressor(max_depth=K,random_state=42)
    regressor_tree_3.fit(X_train,y_train)
    scores_tree.append(regressor_tree_3.score(X_test,y_test))
    y_tree_pred_train_3 = regressor_tree_3.predict(X_train)
    y_tree_pred_test_3 = regressor_tree_3.predict(X_test)

    RMSE_train_3 = mean_squared_error(y_train, y_tree_pred_train_3, squared=False)
    RMSE_test_3 = mean_squared_error(y_test, y_tree_pred_test_3, squared=False)


    rmse_val_train_tree.append(RMSE_train_3)
    rmse_val_test_tree.append(RMSE_test_3)

In [ ]:
curve_train_tree = pd.DataFrame(rmse_val_train_tree)
curve_test_tree = pd.DataFrame(rmse_val_test_tree)

fig = plt.figure(figsize = (15,4))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

ax1.scatter(k_range,scores_tree,marker="o")
ax2.scatter(k_range,rmse_val_train_tree, marker = "o",label="train")
ax2.scatter(k_range,rmse_val_test_tree, marker = "o",label="test")
ax3.plot(curve_train_tree)
ax4.plot(curve_test_tree)

ax1.set_xticks(k_range)
ax2.set_xticks(k_range)

plt.show()

Elijo

In [ ]:
regressor_tree_2=DecisionTreeRegressor(max_depth=4,random_state=42)
regressor_tree_2.fit(X_train,y_train)
y_tree_pred_train_2 = regressor_tree_2.predict(X_train)
y_tree_pred_test_2 = regressor_tree_2.predict(X_test)

In [ ]:
print("Evaluación del árbol de decisiones:","\n")
metricas(y_tree_pred_train_2,y_tree_pred_test_2)

*Elijo el número de profundidad de árbol:* Como referencia usaré 4 como profundidad ya que no se despegan mucho los modelos de training y test y da los resultados de RMSE más bajos.

**CONCLUSIÓN:** Como preferencia mostrandose en los resultados de evaluación, elegiría como principal modelo el de vecinos más cercanos, dando más precisión y similitud entre train y test

**3.7 ¿Qué información no estás usando que podría ayudar al modelo?**

Como se comprobó en el punto 2.1, hay una relación entre el posicionamiento con respecto a zonas de interés y el precio de la vivienda. Si agregamos una columna que establezca un grado de importancia por cercanía a esas zonas, los valores de latitud y longitud podrían ser más relevantes y ayudar a establecer una predicción más precisa con la ayuda de este parámetro.

**3.8 ¿Qué información puede estar demás o repetida?**

Los datos de cuantos ambientes tienen las propiedades podrían no ser necesarios ya que se cuenta con la información del total de ambientes junto a los baños, además de tener una correlatividad demasiado alta con los cuartos lo cual podría sesgar el modelo de predicción.

**Escalado de datos:**
El escalado de datos se realiza cuando se tienen variables
con distinto rango de unidades, lo cual en este dataset contamos
con todas las unidades numéricas en el mismo tipo de unidad, metros,
por lo tanto considero que no es necesario escalar.

**Generación de nuevas variables predictoras: **Esto lo realizaría si no contáramos con la información de barrio de las distintas propiedades, lo cual se podría calcular a partir de la latitud y longitud, pero como no es el caso, esta transformación no es necesario de aplicar.

**La transformación de datos** de variables categóricas a numéricas
en este caso podría resultar útil para que el modelo las pueda utilizar,
por lo tanto se puede proceder con One-Hot-Enconding para tratar con 
las categorías de tipo de propiedad ya que es de tipo ordinal.

El dataset presenta muchos **valores faltantes:**
Estos se tienen que tratar, para esto se debe primero analizar que tipo de
mecanismo es y el posible motivo por el cual es un dato faltante, sea MCAR,MAR
o MNAR.Luego observar con qué se debe proseguir, si eliminar instancias 
o eliminar variables con atributos faltantes o también puede que sea
óptimo imputar.

# Valores faltantes


In [ ]:
import missingno as msno
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from pandas import DataFrame
drive.mount("/content/gdrive")

data_properties_raw = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/DS_Proyecto_01_Datos_Properati.csv", 
                        error_bad_lines=False)

In [ ]:
data_properties_raw.head()

Primero eliminamos las columnas que no aportarían nada al modelo, primero el tipo de anuncio(operation_type) ya que no influyen en la determinación de su valor, luego las fechas del anuncio respectivo(start_date,end_date y created_on) y por último la latitud y longitud(lat,lon) ya que contamos con la información respectiva de  provincia y barrio siendo redundante el país ya que todos los datos son de Argentina también se puede eliminar.

In [ ]:
data_properties_cleaned = data_properties_raw.drop(columns=['start_date','end_date','created_on','operation_type','l1',"lat","lon"])

Repasamos los valores faltantes con este dataset, en este caso sacando el porcentaje de valores faltantes frente al total de las instancias.

In [ ]:
percent_missing = data_properties_cleaned.isna().sum() * 100 / len(data_properties_cleaned)
percent_missing.sort_values(ascending=False)

In [ ]:
msno.bar(data_properties_cleaned)

Gráfico sobre los valores faltantes donde se puede ver que la superficie total y la cubierta son las variables más faltantes, seguido por los baños.

**MCAR,MAR o MNAR:**Ahora analizamos lo aclarado previamente, el motivo por el cual son valores faltantes.

`Superficie total`: Comparar los valores faltantes con una variable numerica, es decir si los valores faltantes dependen del valor de otra variable




In [ ]:
rango_superficie = pd.cut(data_properties_cleaned.surface_total, bins=20, labels=False) 
rango_superficie.value_counts()

**MAR con respecto a `rooms`** 

In [ ]:
msno.matrix(data_properties_cleaned.sort_values('rooms'));

**MAR con respecto a `property_type`**

---




In [ ]:
msno.matrix(data_properties_cleaned.sort_values('property_type'));

**MAR con respecto a `bedrooms`**

In [ ]:
msno.matrix(data_properties_cleaned.sort_values('bedrooms'));

Los valores de bedrooms mas bajos omiten esta información

**MAR con respecto a `l2`**

In [ ]:
msno.matrix(data_properties_cleaned.sort_values('l2'));

**MAR con respecto a `l3`**

In [ ]:
msno.matrix(data_properties_cleaned.sort_values('l3'));

**MNAR:** Analizamos las correlaciones para después hacer una comparativa por si la faltante de valores depende de otros.

In [ ]:
msno.heatmap(data_properties_cleaned)

En el gráfio de correlaciones se puede observar como los valores faltantes de superficie cubierta y superficie total depende uno del otro, lo cual podría llevar a deducir que estos valores faltan por el hecho de desconocer las medidas de la propiedad en general.

In [ ]:
msno.dendrogram(data_properties_cleaned)

**Eliminar valores faltantes**: Ya detectados los posibles motivos y relaciones entre las variables se puede proseguir a imputar y eliminar los valores faltantes. En este paso se proseguiría a eliminar las columnas y filas que posean muchos valores faltantes, en el caso de las columnas no se cuenta con un porcentaje muy grande para considerar esa columna apta para ser eliminada. Por otro lado si se puede eliminar las filas con muchos valores faltantes. Luego comprobamos cuantas instancias se eliminaron.

In [ ]:
data_properties_cleaned_v2 = data_properties_cleaned.copy()

In [ ]:
data_properties_cleaned_v2.dropna(inplace=True) 

In [ ]:
data_properties_cleaned_v2.shape

In [ ]:
print("Instancias con valores faltantes eliminados:",len(data_properties_cleaned.index)-len(data_properties_cleaned_v2.index))

In [ ]:
percent_missing = data_properties_cleaned_v2.isna().sum() * 100 / len(data_properties_cleaned)
percent_missing.sort_values(ascending=False)

# Imputando valores faltantes por media

In [ ]:
corr = data_properties_cleaned_v2.corr()

plt.figure(figsize=(8,8))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
           cmap= 'coolwarm')
plt.xticks(rotation = 45)
plt.yticks(rotation = 45)
plt.show()


Imputamos primero la superficie cubierta con el tipo de propiedad

In [ ]:
media = data_properties_cleaned_v2[["surface_covered","property_type"]].groupby("property_type").agg(pd.Series.mean)


In [ ]:
media = media.to_dict()["surface_covered"]

In [ ]:
data_properties_cleaned_v2.set_index("property_type", inplace=True)

In [ ]:
data_properties_cleaned_v2.surface_covered.fillna(media, inplace=True)

In [ ]:
data_properties_cleaned_v2.reset_index(inplace=True)

Comprobamos que los valores fueron reemplazados

In [ ]:
percent_missing = data_properties_cleaned_v2.isna().sum() * 100 / len(data_properties_cleaned)
percent_missing.sort_values(ascending=False)

Imputamos la superficie total con bathrooms ya que mientras más cuartos haya probablemente tenga más superficie además que rooms no tiene valores faltantes por lo tanto no nos quedarían NaN.

In [ ]:
media = data_properties_cleaned_v2[["surface_total","rooms"]].groupby("rooms").agg(pd.Series.mean)

In [ ]:
media = media.to_dict()["surface_total"]

In [ ]:
data_properties_cleaned_v2.set_index("rooms", inplace=True)

In [ ]:
data_properties_cleaned_v2.surface_total.fillna(media, inplace=True)

Recibe nombre de columna y lo pone como índice para luego llenar los valores nulos con la media.

Reseteamos el index

In [ ]:
data_properties_cleaned_v2.reset_index(inplace=True)

In [ ]:
percent_missing = data_properties_cleaned_v2.isna().sum() * 100 / len(data_properties_cleaned)
percent_missing.sort_values(ascending=False)

#Outliers

Antes de cualquier imputación se deben encontrar los outliers, a continuación se utilizará la regla de las tres sigmas para determinar si es outliers o no,considerando que los valores se encuentren dentro de tres desviaciones estándar de la media.Ya teniendo los outliers, se los marca como NaN así no afectan a la imputación.

In [ ]:
plt.subplot(2, 1, 1)
sns.boxplot(data=data_properties_cleaned_v2,x="surface_total")

plt.subplot(2, 1, 2)
sns.boxplot(data=data_properties_cleaned_v2,x="surface_covered")
plt.show()

In [ ]:
data_properties_cleaned_v3=data_properties_cleaned_v2.copy()

Imputando por media.

In [ ]:
columnas = ["surface_covered", "rooms", "price", "surface_total", "bathrooms"]

for col in columnas:

  desviacion_estandard = data_properties_cleaned_v2[col].std()
  media = data_properties_cleaned_v3[col].mean()

  res = (data_properties_cleaned_v3[col] > media + desviacion_estandard*3) | (data_properties_cleaned_v3[col] < media - desviacion_estandard*3)
  data_properties_cleaned_v3.loc[res,col]=np.nan

In [ ]:
columnas_imputar = ["surface_total", "bathrooms"]

for columna in columnas_imputar:
  data_properties_cleaned_v3[columna].fillna(data_properties_cleaned_v3[columna].median()) 

In [ ]:
data_properties_cleaned_v3.isna().sum() * 100 / data_properties_cleaned_v3.isna().count()

# Codificacion de variables no numericas

Codifico los tipos de propiedad asignandoles un número específico.

Cambio los nombres de las variables l2 y l3 para que sean más fáciles de leer.

In [ ]:
data_properties_cleaned_v3.rename({"l2":"zona","l3":"ciudad"},axis=1,inplace=True)

In [ ]:
dicc = {"Casa":3,"PH":2,"Departamento":1}

In [ ]:
data_properties_cleaned_v3.property_type = data_properties_cleaned_v3.property_type.map(dicc)

In [ ]:
data_properties_cleaned_v3

Codifico los departamentos y ciudades con one-hot-encoding.

Creo un diccionario con las zonas más importantes y los valores que no se encuentran se marcan como NaN.

In [ ]:
dicc = data_properties_cleaned_v3.zona.value_counts().iloc[:].to_dict()
dicc = { k:k for k in dicc.keys() }

Reemplazo la columna de zona ya que usaremos one-hot encoding y luego reemplazo los NaN en una categoría de Otros.

In [ ]:
dicc

In [ ]:
data_properties_cleaned_v3.zona = data_properties_cleaned_v3.zona.map(dicc)


In [ ]:
#data_properties_cleaned_v3.fillna("Otros", inplace=True)

In [ ]:
data_properties_cleaned_v3 = pd.get_dummies(data_properties_cleaned_v3, columns=["zona"])

In [ ]:
data_properties_cleaned_v3

Sólo queda ciudad como una variable categórica, para pasarla a numérica se hará una codificación por metro cuadrado.

In [ ]:
data_properties_cleaned_v4 = data_properties_cleaned_v3.copy()

In [ ]:
data_properties_cleaned_v4.dropna(inplace=True)

In [ ]:
data_properties_cleaned_v4.surface_total.value_counts()

In [ ]:
data_properties_cleaned_v4["m2"]  = data_properties_cleaned_v3.price /data_properties_cleaned_v3.surface_total

In [ ]:
data_properties_cleaned_v4.head()

Determino cuanto vale el metro cuadrado en la ciudad, calculando la media.

In [ ]:
media = data_properties_cleaned_v4[["m2","ciudad"]].groupby("ciudad").agg(pd.Series.mean)
media = media.to_dict()["m2"]
media

Reemplazo la columna ciudad codificandolo con la media de m2.

In [ ]:
data_properties_cleaned_v4.ciudad = data_properties_cleaned_v4.ciudad.map(media)

Elimino la columna de m2 ya que esta supone un precio que luego tomaría el modelo para entrenarse y daría una precisión alta.

In [ ]:
data_properties_cleaned_v4.drop(columns=["m2"], inplace=True)

In [ ]:
data_properties_cleaned_v4.head()

# Escalador de datos

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = data_properties_cleaned_v4.drop(columns=["price","currency","title","description"])
y = data_properties_cleaned_v4.price


In [ ]:
scl = StandardScaler()
scl.fit(X)

In [ ]:
X_scl = scl.transform(X)
X_scl.shape

#### Modelado

Realizamos el modelado con los datos escalados hechos.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_scl, y)

In [ ]:
model = RandomForestRegressor(100, criterion="mse", max_depth=4, max_features=6)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
mean_absolute_error(model.predict(x_train), y_train)

In [ ]:
mean_absolute_error(model.predict(x_test), y_test)

# Regresión lineal

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import linear_model

Evaluador

In [ ]:
def cross_val(model):
    pred = cross_val_score(model, X_scl, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train) 

In [ ]:
y_predicted = regr.predict(x_test)
cross_val(regr)
print_evaluate(y_test,y_predicted)

Ridge


In [ ]:
reg = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))
reg.fit(x_train, y_train) 

In [ ]:
y_predicted_r = reg.predict(x_test)
cross_val(reg)
print_evaluate(y_test,y_predicted_r)

Lasso


In [ ]:
reg_l = linear_model.Lasso(alpha=2)
reg_l.fit(x_train, y_train) 

In [ ]:
y_predicted_l = reg_l.predict(x_test)
cross_val(reg_l)
print_evaluate(y_test,y_predicted_l)

#RANDOM SEARCH con GRADIENT BOOSTING


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [ ]:
model_GB = GradientBoostingRegressor()
parameters = {'learning_rate': sp_randFloat(),
                  'subsample'    : sp_randFloat(),
                  'n_estimators' : sp_randInt(100, 1000),
                  'max_depth'    : sp_randInt(4, 10)
                 }

In [ ]:
randm_src = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                               cv = 2, n_iter = 10, n_jobs=-1)
randm_src.fit(x_train, y_train)

print(" Results from Random Search " )
print("\n The best estimator across ALL searched params:\n", randm_src.best_estimator_)
print("\n The best score across ALL searched params:\n", randm_src.best_score_)
print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)

In [ ]:
grboosting = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.04081003063069022,
                          loss='ls', max_depth=9, max_features=None,
                          max_leaf_nodes=None, min_impurity_decrease=0.0,
                          min_impurity_split=None, min_samples_leaf=1,
                          min_samples_split=2, min_weight_fraction_leaf=0.0,
                          n_estimators=592, n_iter_no_change=None,
                          presort='deprecated', random_state=None,
                          subsample=0.169613226680299, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
grboosting.fit(x_train, y_train) 

In [ ]:
print("Cross Validation:",cross_val(grboosting))
print_evaluate(y_test,y_predictedgbl)

In [ ]:
y_predictedgbl_test = grboosting.predict(x_test)
y_predictedgbl_train = grboosting.predict(x_train)
metricas(y_predictedgbl_train,y_predictedgbl_test)

# RANDOM FOREST REGRESSION con GRID SEARCH



In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid={'bootstrap':[True],'max_depth':[5,10,None],'max_features':['auto','log2'],'n_estimators':[5,6,7,8,9,10,11,12,13,15]}

In [ ]:
rfr = RandomForestRegressor(random_state = 1)

In [ ]:
g_search = GridSearchCV(estimator = rfr, param_grid = param_grid,cv = 3,n_jobs = 1,verbose = 0,return_train_score=True)

In [ ]:
g_search.fit(x_train, y_train);
print(g_search.best_params_)

In [ ]:
rfr_gs = RandomForestRegressor(bootstrap=True, max_depth= None, max_features= "log2", n_estimators= 15)

In [ ]:
rfr_gs.fit(x_train, y_train) 

In [ ]:
y_predictedrgs = rfr_gs.predict(x_test)
y_predictedrgs_train = rfr_gs.predict(x_train)
print("Cross Validation:",cross_val(rfr_gs))
print_evaluate(y_test,y_predictedrgs)

In [ ]:
metricas(y_predictedrgs_train,y_predictedrgs)